In [1]:
import minai.datasets as minds
import minai.data as mind
import minai.sampler as mins

In [2]:
dsd = minds.hf_load("fashion_mnist")

Found cached dataset fashion_mnist (/home/nblzv/.cache/huggingface/datasets/fashion_mnist/fashion_mnist/1.0.0/0a671f063342996f19779d38c0ab4abef9c64f757b35af8134b331c294d7ba48)


  0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
dl = mind.DataLoader.hf(dsd["train"])
dl

DataLoader(ds=Dataset({
    features: ['image', 'label'],
    num_rows: 60000
}),
  CMTO(SIO(batch_size=64, shuffle=True, drop_last=False),
    getitem_func=Dataset.__getitem__,
    collate_func=HFCollate.__call__,
    num_workers=15,
    max_available_batches=2,
    chunk_size_per_thread=4,
    is_hf_ds=True))

In [12]:
it = iter(dl)
a = next(it)